In [1]:
# @title Data retrieval
import os, requests
#download data 
fname = 'motor_imagery.npz'
url = "https://osf.io/ksqv8/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [2]:
# @title Install packages (`nilearn`, `nimare`. `duecredit`), import `matplotlib` and set defaults
# install packages to visualize brains and electrode locations
!pip install nilearn --quiet
!pip install nimare --quiet
!pip install duecredit --quiet

from matplotlib import rcParams
from matplotlib import pyplot as plt
import pandas as pd 

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

     |████████████████████████████████| 9.6 MB 4.9 MB/s 
     |████████████████████████████████| 13.3 MB 5.4 MB/s 
     |████████████████████████████████| 3.2 MB 37.7 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 179 kB 28.5 MB/s 


In [10]:
# @title Data loading
import numpy as np
from pickle import TRUE
alldat = np.load(fname, allow_pickle=True)['dat']

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx.
dat1 = alldat[0][0]
dat2 = alldat[0][1]
# get trial number
# print(len(dat1['stim_id']))

#get trial number
# print(alldat.shape)

In [11]:
# function to get subsets of data
def get_sub_dat(alldata, sub_id, exp_id):
  return alldata[sub_id][exp_id]

In [24]:
def dftimewin(dat):
  data = {'t_on': dat['t_on'], 't_off': dat['t_off'], 'stim_id': dat['stim_id']}
  dftime = pd.DataFrame(data)
  return dftime


In [21]:
def dft(dat, channel_n = 46):
  #get dft
  a = dat['V']
  epoch_id=[]
  for i in range(channel_n):
    epoch_id.append(i+1)
  epoch_id_array = np.array(epoch_id)
  df = pd.DataFrame(a, columns=epoch_id)
  dft = df.T
  return dft

In [26]:
#generate X for SVM
def slice_chan_dat(dft,dftime,channel_i):
  chan_i = dft.loc[channel_i]
  chan_array = []
  for j in range (0, len(dftime)):
    chan_array.append(chan_i[dftime["t_on"][j]: dftime["t_off"][j]])
  # print(chan_array.shape)
  # ar_chan_i = chan_array.values
  result = np.array([np.array(xi) for xi in chan_array])
  return result

In [30]:
#generate y for SVM
def get_y_val(dftime):
  y = dat["stim_id"].values
  return y.shape

In [28]:
#get data from subject 0, experiment 0
dat = get_sub_dat(alldat, 0, 0)
dft = dft(dat, channel_n = 46)
dftime = dftimewin(dat)
# get X and y input for the SVM
X = slice_chan_dat(dft,dftime,1)
y = get_y_val(dftime)

array([[-0.1311  , -0.0793  , -0.00683 , ..., -0.00904 , -0.09314 ,
        -0.183   ],
       [ 1.348   ,  1.363   ,  1.373   , ..., -0.1691  , -0.03354 ,
         0.148   ],
       [-0.2012  , -0.1841  , -0.1384  , ..., -0.00956 ,  0.1858  ,
         0.05383 ],
       ...,
       [-0.006638,  0.0105  , -0.0494  , ...,  2.293   ,  2.223   ,
         2.326   ],
       [-0.8135  , -0.777   , -0.7275  , ...,  0.01761 , -0.04407 ,
        -0.1273  ],
       [-0.4937  , -0.3262  , -0.2394  , ...,  0.664   ,  0.6     ,
         0.4048  ]], dtype=float16)